In [10]:
WITH CAMPANHAS AS (
    SELECT TOP 100 *
    FROM LoginRawData Log WITH (NOLOCK)
    INNER JOIN Users U ON U.Agentld = Log.Agentld
    WHERE U.[Login] = 'beatriz_nunes'
    AND CAST(StartLogin AS DATE) = '2024-07-30'
    ORDER BY StartLogin
)
SELECT *
FROM CAMPANHAS;


: Msg 8156, Level 16, State 1, Line 1
The column 'Agentld' was specified multiple times for 'CAMPANHAS'.

Total execution time: 00:00:00.088

In [3]:
SET NOCOUNT ON;
SET ANSI_WARNINGS OFF;

DECLARE @dinicio DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), DATEADD(DAY, -1, GETDATE()) , 121) + ' 00:00:00');
DECLARE @dfim DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), DATEADD(DAY, -1, GETDATE()), 121) + ' 23:59:00');

DECLARE @ligacoes TABLE (
    DATA DATE,
    LOGIN VARCHAR(30),
    TotalAgentCalls INT,
    TotalAgentTime INT,
    PreviewTime INT,
    PreviewQt INT,
    ActiveTime INT,
    WrapTime INT
);

INSERT INTO @ligacoes
SELECT 
    CAST(DATEADD(HOUR, -3, BeginTimePeriodDt) AS DATE) AS DATA,
    user_id AS LOGIN,
    SUM([TotalAgentCalls]),
    SUM([TotalAgentTime]),
    SUM([PreviewTime]),
    SUM(CASE WHEN [PreviewTime] > 0 THEN 1 ELSE 0 END) AS PreviewQt,
    SUM([ActiveTime]),
    SUM([WrapTime])
FROM 
    summary_epro..[MediaAgentSummary] WITH (NOLOCK)
WHERE 
    DATEADD(HOUR, -3, BeginTimePeriodDt) BETWEEN @dinicio AND @dfim
GROUP BY 
    CAST(DATEADD(HOUR, -3, BeginTimePeriodDt) AS DATE),
    user_id;

-- Criação da tabela temporária
DECLARE @work TABLE (
    DATA DATE,
    ID_DISCADOR_OPERADOR INT,
    LOGIN_DISCADOR_OPERADOR VARCHAR(50),
    NOME_DISCADOR_OPERADOR VARCHAR(150),
    LOGIN_DISCADOR_SUPERVISOR VARCHAR(200),
    LOGIN TIME,
    LOGOUT TIME,
    QTD_LOGIN INT,
    QTD_LOGOUT INT,
    TEMPO_LOGADO INT,
    TEMPO_DISPONIVEL INT,
    TEMPO_IDLE INT,
    CAMPANHA VARCHAR(200),
    ID_CAMPANHA INT,
    ID_NOVO INT
);

-- Inserção de dados na tabela temporária
INSERT INTO @work
SELECT 
    CAST(DATEADD(HOUR, -3, t1.LoginDt) AS DATE) AS DATA,
    t1.WorkGroup_Id AS ID_DISCADOR_OPERADOR,
    t1.User_Id AS LOGIN_DISCADOR_OPERADOR,
    USR.User_F_Name + ' ' + USR.User_L_Name AS NOME_DISCADOR_OPERADOR,
    WK.Workgroup_Name AS LOGIN_DISCADOR_SUPERVISOR,
    MIN(DATEADD(HOUR, -3, t1.LoginDt)) AS LOGIN,
    MAX(DATEADD(HOUR, -3, ISNULL(t1.LogoutDt, @dfim))) AS LOGOUT,
    COUNT(t1.LoginDt) AS QTD_LOGIN,
    COUNT(t1.LogoutDt) AS QTD_LOGOUT,
    SUM(CASE 
            WHEN t1.LogoutDt IS NOT NULL THEN t1.LoginTime
            ELSE DATEDIFF(SECOND, t1.LoginDt, ISNULL(t1.LogoutDt, @dfim))
        END) AS TEMPO_LOGADO,
    SUM(t1.GapTime) AS TEMPO_DISPONIVEL,
    SUM(t1.IdleTime) AS TEMPO_IDLE,
    SER.Service_c AS CAMPANHA,
    SER.Service_Id AS ID_CAMPANHA,
    DENSE_RANK() OVER (PARTITION BY t1.User_Id ORDER BY SER.Service_c) AS ID_NOVO
FROM 
    summary_epro..[AgentSignInSummary] t1 WITH (NOLOCK)
    INNER JOIN config_epro..[Workgroup] wk WITH (NOLOCK) ON t1.Workgroup_Id = wk.Workgroup_Id
    INNER JOIN config_epro..[Service_Group] AS SERG WITH (NOLOCK) ON SERG.User_Id = t1.User_Id
    INNER JOIN config_epro..[Service] AS SER WITH (NOLOCK) ON SER.Service_Id = SERG.Service_Id
    JOIN config_epro..[Users] AS USR WITH (NOLOCK) ON SERG.User_Id = USR.User_Id
WHERE 
    DATEADD(HOUR, -3, t1.LoginDt) BETWEEN @dinicio AND @dfim
GROUP BY 
    CAST(DATEADD(HOUR, -3, t1.LoginDt) AS DATE),
    t1.User_Id,
    t1.WorkGroup_Id,
    wk.Workgroup_Name,
    USR.User_F_Name + ' ' + USR.User_L_Name,
    SER.Service_c,
    SER.Service_Id;

-- Consulta dos dados na tabela temporária
SELECT 
    w.DATA,
    w.ID_CAMPANHA,
    w.CAMPANHA,
    w.ID_DISCADOR_OPERADOR,
    w.LOGIN_DISCADOR_OPERADOR,
    w.NOME_DISCADOR_OPERADOR,
    w.LOGIN_DISCADOR_SUPERVISOR,
    w.LOGIN,
    w.LOGOUT,
    w.QTD_LOGIN,
    w.QTD_LOGOUT,
    ISNULL(l.TotalAgentCalls, 0) AS ATENDIDAS,
    w.TEMPO_LOGADO,
    DATEDIFF(SECOND, w.LOGIN, w.LOGOUT) - IIF(DATEDIFF(SECOND, w.LOGIN, w.LOGOUT) - w.TEMPO_LOGADO > 0, w.TEMPO_LOGADO, 0) AS TEMPO_DESLOGADO,
    ISNULL(l.ActiveTime, 0) + ISNULL(l.WrapTime, 0) + w.TEMPO_IDLE AS TEMPO_DISPONIVEL,
    w.TEMPO_LOGADO - (ISNULL(l.ActiveTime, 0) + ISNULL(l.WrapTime, 0) + w.TEMPO_IDLE) AS TEMPO_PAUSA,
    ISNULL(l.ActiveTime, 0) AS TEMPO_FALADO,
    ISNULL(l.WrapTime, 0) AS TEMPO_POS,
    w.TEMPO_IDLE,
    ISNULL(l.ActiveTime, 0) / ISNULL(l.TotalAgentCalls, 0) AS TMA,
    ISNULL(l.WrapTime, 0) / ISNULL(l.TotalAgentCalls, 0) AS TMP,
    w.TEMPO_IDLE / ISNULL(l.TotalAgentCalls, 0) AS TME,
    ISNULL(l.ActiveTime, 0) / ISNULL(l.TotalAgentCalls, 0) + ISNULL(l.WrapTime, 0) / ISNULL(l.TotalAgentCalls, 0) AS TMO,
    '10.12.101.21' AS INSTANCIA,
    'detail_epro' AS BANCO,
    'ASPECT' AS ORIGEM
FROM 
    @work w
    LEFT JOIN @ligacoes l ON (
        l.DATA = w.DATA AND l.LOGIN = w.LOGIN_DISCADOR_OPERADOR
    )
WHERE 
    w.ID_NOVO = 1
    AND w.LOGIN_DISCADOR_OPERADOR != 'rstaspectagent'
    AND w.DATA = CAST(@dinicio AS DATE);


Commands completed successfully.

Total execution time: 00:00:16.746

DATA,ID_CAMPANHA,CAMPANHA,ID_DISCADOR_OPERADOR,LOGIN_DISCADOR_OPERADOR,NOME_DISCADOR_OPERADOR,LOGIN_DISCADOR_SUPERVISOR,LOGIN,LOGOUT,QTD_LOGIN,QTD_LOGOUT,ATENDIDAS,TEMPO_LOGADO,TEMPO_DESLOGADO,TEMPO_DISPONIVEL,TEMPO_PAUSA,TEMPO_FALADO,TEMPO_POS,TEMPO_IDLE,TMA,TMP,TME,TMO,INSTANCIA,BANCO,ORIGEM
2024-07-31,9,MF_TIM_TELECOB,9,TIMADRIANACAMPOS,ADRIANA OLIVEIRA CAMPOS,TIM_COB_SUP_JULIANA_VON_PAUMGARTTEN_DE_ARAUJO,08:06:49.1120000,14:32:45.6100000,1,1,218,23156,23156,19702,3454,9877,1409,8416,45,6,38,51,10.12.101.21,detail_epro,ASPECT
2024-07-31,9,MF_TIM_TELECOB,10,TIMADRIANACLE,ADRIANA CLEMENTE FERREIRA,TIM_COB_SUP_MIRIAN_MOTA_TORRES,14:21:36.9190000,21:06:30.1490000,1,1,175,24293,1,21579,2714,8591,1090,11898,49,6,67,55,10.12.101.21,detail_epro,ASPECT
2024-07-31,10,MF_TIM_LIVE_TC,10,TIMALAN5059,ALAN CASTRO DA SILVA,TIM_COB_SUP_MIRIAN_MOTA_TORRES,14:22:56.7850000,21:06:30.1610000,1,1,153,24213,1,21592,2621,2955,1425,17212,19,9,112,28,10.12.101.21,detail_epro,ASPECT
2024-07-31,15,TIM_CALLBACK_MANUAL,5,TIMALCILENE7858,AUCILENE MENDES DE SOUSA,TIM_COB_SUP_HELDER_RODRIGUES_MACIEL,14:28:43.5050000,21:06:30.2120000,1,1,154,23867,23867,23180,687,7744,1056,14380,50,6,93,56,10.12.101.21,detail_epro,ASPECT
2024-07-31,39,TIM_CHURN,4,TIMALESSANDRA441,ALESSANDRA DOS SANTOS MARTINS,TIM_COB_SUP_PAULA_VANESSA_SOUZA_DA_SILVA,14:23:22.2600000,21:06:30.1670000,1,1,137,24188,24188,20572,3616,3643,1096,15833,26,8,115,34,10.12.101.21,detail_epro,ASPECT
2024-07-31,39,TIM_CHURN,4,TIMALEXIA2281,ALEXIA MARIA DE SOUSA LEITE,TIM_COB_SUP_PAULA_VANESSA_SOUZA_DA_SILVA,07:40:40.2380000,14:20:03.9560000,1,1,160,23964,23963,20660,3304,2622,1466,16572,16,9,103,25,10.12.101.21,detail_epro,ASPECT
2024-07-31,9,MF_TIM_TELECOB,7,TIMALISSON6134,ALISSON JOSE COSTA BASTOS,TIM_COB_SUP_KARYANNE_SUELLEN_LIMA_DE_OLIVEIRA,07:42:58.7860000,14:19:51.3350000,2,2,203,23621,192,20706,2915,8200,2004,10502,40,9,51,49,10.12.101.21,detail_epro,ASPECT
2024-07-31,9,MF_TIM_TELECOB,4,TIMANA1689,ANA LIVIA MENDES DE SOUSA,TIM_COB_SUP_PAULA_VANESSA_SOUZA_DA_SILVA,14:24:19.4430000,21:06:30.1840000,1,1,177,24131,24131,21056,3075,7021,1613,12422,39,9,70,48,10.12.101.21,detail_epro,ASPECT
2024-07-31,9,MF_TIM_TELECOB,5,TIMANA2945,ANA JULIA SILVA DOS SANTOS DIAS,TIM_COB_SUP_HELDER_RODRIGUES_MACIEL,14:26:22.1480000,21:06:30.2590000,3,3,164,23375,633,20621,2754,4025,1720,14876,24,10,90,34,10.12.101.21,detail_epro,ASPECT
2024-07-31,9,MF_TIM_TELECOB,4,TIMANA3110,ANA CLAUDIA SANTOS DA CONCEICAO,TIM_COB_SUP_PAULA_VANESSA_SOUZA_DA_SILVA,14:23:38.3350000,21:06:30.1730000,1,1,177,24172,24172,20295,3877,9854,1234,9207,55,6,52,61,10.12.101.21,detail_epro,ASPECT


In [36]:
--10455
SET NOCOUNT ON;
SET ANSI_WARNINGS OFF;

DECLARE @dinicio DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), GETDATE(), 121) + ' 00:00:00');
DECLARE @dfim DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), GETDATE(), 121) + ' 23:59:00');

-- Temp table to store call data
DECLARE @Ligacoes TABLE (
    Agentld INT,
    TEMPO_FALADO INT,
    TEMPO_IDLE INT, 
    TEMPO_POS INT,
    TEMPO_DISPONIVEL INT,
    ATENDIDAS INT
);

-- Insert into @Ligacoes
INSERT INTO @Ligacoes 
SELECT 
    Agent.Agentld,
    SUM(CASE WHEN [Description] in ('Talking') THEN DATEDIFF(SECOND, Agent.StartState, Agent.EndState) ELSE 0 END) AS TEMPO_FALADO,
    SUM(CASE WHEN [Description] IN ('Idle') THEN DATEDIFF(SECOND, Agent.StartState, Agent.EndState) ELSE 0 END) AS TEMPO_IDLE,
    SUM(CASE WHEN [Description] IN ('Wrap') THEN DATEDIFF(SECOND, Agent.StartState, Agent.EndState) ELSE 0 END) AS TEMPO_POS,
    SUM(CASE WHEN [Description] IN ('Wrap','Idle','Talking') THEN DATEDIFF(SECOND, Agent.StartState, Agent.EndState) ELSE 0 END) AS TEMPO_DISPONIVEL,
    SUM(CASE WHEN Callld != 0 THEN 1 ELSE 0 END) AS ATENDIDAS
FROM AgentStateRawData Agent WITH (NOLOCK)
INNER JOIN Info_AgentStatus Info ON Info.AgentStatusId = Agent.AgentStatus
WHERE CAST(StartState AS DATE) = CAST('2024-07-31' AS DATE)
GROUP BY Agent.Agentld;

SELECT * FROM @Ligacoes 
--10510

Commands completed successfully.

Total execution time: 00:00:27.559

Agentld,TEMPO_FALADO,TEMPO_IDLE,TEMPO_POS,TEMPO_DISPONIVEL,ATENDIDAS
1142,2769,34479,1351,38599,432
1146,12340,15738,3716,31794,1940
1256,6452,6146,5343,17941,808
1259,14499,24516,2739,41754,3478
1263,0,34836,0,34836,0
1295,0,33749,0,33749,0
1332,16,49693,20,49729,6
1338,13609,19794,3676,37079,2500
1339,13350,14295,2047,29692,1870
1350,103,35961,45,36109,6


In [58]:
SELECT TOP 1000 *
    FROM AgentStateRawData Agent WITH (NOLOCK)
    INNER JOIN Info_AgentStatus Info ON Info.AgentStatusId = Agent.AgentStatus
    WHERE CAST(StartState AS DATE) = CAST( DATEADD(DAY,-1,GETDATE()) AS DATE)
    --AND Info.[Description] LIKE '%Pause%'
   AND Agentld = 6677
    ORDER BY Agent.AgentStateRawDatalD ASC

Commands completed successfully.

Total execution time: 00:00:04.428

Agentld,Campaignld,AgentStatus,Reason,StartState,EndState,Callld,CallidTelecom,AgentStateRawDatalD,InsertTime,AgentStatusId,Description
6677,71,1,0,2024-07-31 12:09:16.103,2024-07-31 12:09:32.007,0,,683969252,1381763862,1,Idle


In [56]:
SET NOCOUNT ON;
SET ANSI_WARNINGS OFF;

DECLARE @dinicio DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), DATEADD(DAY, - 1, GETDATE()), 121) + ' 00:00:00');
DECLARE @dfim DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), DATEADD(DAY, - 1, GETDATE()), 121) + ' 23:59:00');
DECLARE @AuxiliarCampanhas TABLE (
	Agentld INT
	,Campaignld INT
	,INDICE INT
	);

INSERT INTO @AuxiliarCampanhas
SELECT DISTINCT Agentld
	,Campaignld
	,DENSE_RANK() OVER (
		PARTITION BY LOG.Agentld ORDER BY LOG.Campaignld
		) AS ID_NOVO
FROM LoginRawData LOG WITH (NOLOCK)
WHERE CAST(StartLogin AS DATE) = CAST(@dinicio AS DATE);


DECLARE @WORK TABLE (
	DATA DATE
	,ID_DISCADOR_OPERADOR INT
	,LOGIN_DISCADOR_OPERADOR VARCHAR(40)
	,NOME_DISCADOR_OPERADOR VARCHAR(80)
	,LOGIN_DISCADOR_SUPERVISOR VARCHAR(80)
	,LOGIN TIME
	,LOGOUT TIME
	,QTD_LOGIN INT
	,QTD_LOGOUT INT
	,TEMPO_LOGADO INT
	,TEMPO_DESLOGADO INT
    -- , ID_NOVO INT
    -- , ID_CAMPANHA INT
    -- , CAMPANHA VARCHAR(100)
	);


;WITH Working as (
SELECT CAST(StartLogin AS DATE) AS DATA
	,A.Agentld AS ID_DISCADOR_OPERADOR
	,E.LOGIN AS LOGIN_DISCADOR_OPERADOR
	,E.UserName AS NOME_DISCADOR_OPERADOR
	,C_user.UserName AS LOGIN_DISCADOR_SUPERVISOR
	,MIN(CONVERT(TIME, StartLogin, 108)) AS LOGIN
	,MAX(CONVERT(TIME, EndLogin, 108)) AS LOGOUT
	,COUNT(StartLogin) AS QTD_LOGIN
	,COUNT(EndLogin) AS QTD_LOGOUT
	,DATEDIFF(SECOND, MIN(StartLogin), MAX(EndLogin)) AS TEMPO_LOGADO
	,CASE 
		WHEN DATEDIFF(SECOND, LAG(EndLogin) OVER (
					PARTITION BY E.LOGIN ORDER BY StartLogin
						,EndLogin
					), StartLogin) >= 0
			THEN DATEDIFF(SECOND, LAG(EndLogin) OVER (
						PARTITION BY E.LOGIN ORDER BY StartLogin
							,EndLogin
						), StartLogin)
		ELSE 0
		END AS TEMPO_DESLOGADO
	-- ,DENSE_RANK() OVER (
	-- 	PARTITION BY A.Agentld ORDER BY A.Campaignld
	-- 	) AS ID_NOVO,
    --     A.Campaignld as ID_CAMPANHA,
    --     c.[Description] as CAMPANHA
FROM LoginRawData A
INNER JOIN Users E WITH (NOLOCK) ON A.Agentld = E.Agentld
INNER JOIN AgentSupervisor Sup WITH (NOLOCK) ON Sup.AgentId = E.Agentld
INNER JOIN Users C_user WITH (NOLOCK) ON Sup.SupervisorId = C_user.Agentld
--INNER JOIN Campaign C WITH (NOLOCK) ON C.Campaignld = A.Campaignld
WHERE StartLogin BETWEEN @dinicio
		AND @dfim
GROUP BY CAST(StartLogin AS DATE)
	,A.Agentld
	,E.LOGIN
	,E.UserName
	,C_user.UserName ,StartLogin, EndLogin

)


INSERT INTO @WORK

SELECT  
    W.[DATA]
    ,W.ID_DISCADOR_OPERADOR
    ,W.LOGIN_DISCADOR_OPERADOR
    ,W.NOME_DISCADOR_OPERADOR
    ,W.LOGIN_DISCADOR_SUPERVISOR
    ,MIN(W.[LOGIN]) AS LOGIN
    ,MAX(W.LOGOUT) AS LOGOUT 
    ,SUM(W.QTD_LOGIN) AS QTD_LOGIN
    ,SUM(QTD_LOGOUT) AS QTD_LOGOUT
    ,DATEDIFF(SECOND, MIN(W.[LOGIN]),MAX(W.LOGOUT)) AS TEMPO_LOGADO
    ,SUM(TEMPO_DESLOGADO) AS TEMPO_DESLOGADO

FROM Working W 

GROUP BY W.[DATA]
    ,W.ID_DISCADOR_OPERADOR
    ,W.LOGIN_DISCADOR_OPERADOR
    ,W.NOME_DISCADOR_OPERADOR
    ,W.LOGIN_DISCADOR_SUPERVISOR;


SELECT * FROM @WORK

Commands completed successfully.

Total execution time: 00:00:00.434

DATA,ID_DISCADOR_OPERADOR,LOGIN_DISCADOR_OPERADOR,NOME_DISCADOR_OPERADOR,LOGIN_DISCADOR_SUPERVISOR,LOGIN,LOGOUT,QTD_LOGIN,QTD_LOGOUT,TEMPO_LOGADO,TEMPO_DESLOGADO
2024-07-31,1142,nario_nathanael,NARIO NATHANAEL PEREIRA DE SOUSA,TATIANE VASCONCELOS,08:57:03.2033333,16:11:44.4066667,9,9,26081,370
2024-07-31,1146,raquel_cristine,RAQUEL CRISTINE SOARES DA SILVA,TATIANE VASCONCELOS,07:55:30.8966667,15:13:24.4666667,5,5,26274,0
2024-07-31,1256,samara_silva,SAMARA SILVA MALAQUIAS,TATIANE VASCONCELOS,07:59:57.0866667,12:03:11.9766667,4,4,14594,0
2024-07-31,1259,alinara_arruda,ALINARA ARRUDA RIBEIRO,TATIANE VASCONCELOS,07:59:16.9866667,15:12:51.4666667,8,8,26015,0
2024-07-31,1295,jessica_frida,JESSICA FRIDA PAIVA VASCONCELOS,TATIANE VASCONCELOS,08:40:18.2400000,17:06:38.7166667,7,7,30380,7675
2024-07-31,1332,maria_michele,MARIA MICHELE DA COSTA DE ALMEIDA,TATIANE VASCONCELOS,08:57:13.5166667,16:14:09.3600000,4,4,26216,0
2024-07-31,1338,tayane_braz,TAYANE LINO BRAZ,TATIANE VASCONCELOS,07:56:23.3833333,15:12:39.6166667,5,5,26176,0
2024-07-31,1339,thais_melo,THAIS MELO UCHOA,TATIANE VASCONCELOS,08:15:01.3600000,14:19:20.6100000,10,10,21859,56
2024-07-31,1350,tania_amorim,TANIA MARIA AMORIM,RAIZA MATEUS BRANDAO,07:59:32.5800000,15:16:25.4266667,3,3,26213,0
2024-07-31,1354,maria_eduarda,MARIA EDUARDA RIBEIRO LUCAS,RAIZA MATEUS BRANDAO,09:27:10.0133333,15:19:37.7433333,2,2,21147,0


In [28]:
DECLARE @dinicio DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), DATEADD(DAY, -1, GETDATE()), 121) + ' 00:00:00');

DECLARE @AuxiliarCampanhas TABLE(
    Agentld INT,
    Campaignld INT,
    INDICE INT
);

INSERT INTO @AuxiliarCampanhas
SELECT DISTINCT 
    Agentld,
    Campaignld,
    ROW_NUMBER() OVER (PARTITION BY LOG.Agentld ORDER BY LOG.Agentld DESC) AS INDICE
FROM LoginRawData LOG WITH (NOLOCK)
WHERE CAST(StartLogin AS DATE) = CAST(@dinicio AS DATE);

SELECT * FROM @AuxiliarCampanhas
WHERE INDICE = 1


Commands completed successfully.

Total execution time: 00:00:00.165

Agentld,Campaignld,INDICE
1142,57,1
1146,328,1
1256,461,1
1259,276,1
1263,71,1
1295,58,1
1332,71,1
1338,328,1
1339,63,1
1350,86,1
